In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA5/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
5249,65.281420,0.789293,16.217920,1.320853,3.511767,0.119361,2.351234,3.621155,3.227928,3.559070
5250,65.281119,0.789310,16.217923,1.320933,3.511766,0.119357,2.351381,3.621239,3.227930,3.559043
5251,65.280820,0.789327,16.217926,1.321012,3.511765,0.119353,2.351527,3.621323,3.227931,3.559017
5252,65.280524,0.789343,16.217929,1.321089,3.511766,0.119349,2.351670,3.621405,3.227933,3.558991
5253,65.280232,0.789360,16.217932,1.321165,3.511768,0.119345,2.351812,3.621487,3.227934,3.558965


In [11]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:180000]



In [12]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA5/area5_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA5/area5_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA5/area5_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA5/area5_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [16]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_3_results_excel.txt")
df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_4_results_excel.txt")

In [17]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [18]:
df_final

,Q,P,K
SAMPLE,,,
5249,19.670704,42.544448,23.762197
5250,19.670168,42.544842,23.762018
5251,19.669611,42.545239,23.761899
5252,19.669054,42.545685,23.761724
5253,19.668512,42.546082,23.761606
...,...,...,...
363036,27.915609,36.080022,27.911100
363037,27.915520,36.080044,27.911091
363038,27.915388,36.080143,27.911091


## QAPF classification

In [19]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [20]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [21]:
df_final.to_excel("../_CIPW/CIPW/AREA5/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA5/QAPF_counts.xlsx")

-----

In [23]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=[0], usecols = [0, 1, 2, 3, 4, 29])

In [24]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,-5.499676e+08,4.875427e+08,1.0,W,2.0,52.526539,17.979983,22.933498,monzo granite
2,-3.603806e+08,-3.078196e+08,1.0,W,1.0,45.821201,19.425633,32.374005,monzo granite
3,-3.886908e+08,-8.016822e+08,1.0,W,3.0,50.716606,6.475010,37.375668,syeno granite
4,-3.660122e+08,-6.857011e+08,1.0,W,3.0,41.397182,23.234657,27.592905,monzo granite
5,-2.174685e+08,-6.174700e+08,1.0,W,4.0,54.609970,18.013528,19.746868,monzo granite
...,...,...,...,...,...,...,...,...,...
4654,-5.434578e+08,2.107201e+08,1.0,W,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
4655,-2.908713e+08,-7.509303e+08,1.0,W,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
4656,-5.490291e+08,1.538498e+08,1.0,W,2.0,0.000000,52.355599,12.039131,monzodiorite monzogabbro
4658,-3.958005e+08,-7.167064e+08,1.0,W,3.0,0.000000,71.436466,10.147088,monzodiorite monzogabbro


In [25]:
QAPF_control_area5 = QAPF_control[QAPF_control['area'] ==5]
QAPF_control_area5.to_excel("../_CIPW/CIPW/AREA5/QAPF_control_points.xlsx", index=True)

In [27]:
QAPF_control_area5

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
43,422823.000139,7.525443e+06,1.0,W,5.0,34.500855,34.066073,28.925753,monzo granite
67,451013.431499,7.286869e+06,1.0,W,5.0,34.432877,46.873949,15.788882,granodiorite
111,594545.985094,7.366095e+06,1.0,W,5.0,40.740791,23.591596,31.152534,monzo granite
134,403945.076840,7.527911e+06,1.0,W,5.0,39.354721,25.541001,30.579271,monzo granite
181,401216.148466,7.529882e+06,1.0,W,5.0,38.729450,26.377354,30.528362,monzo granite
...,...,...,...,...,...,...,...,...,...
4562,575266.330715,7.185258e+06,1.0,W,5.0,5.311582,60.423371,16.198088,quartz monzodiorite\nquartz monzogabbro
4594,604290.081053,7.364584e+06,1.0,W,5.0,9.106708,48.239744,17.645349,quartz monzodiorite\nquartz monzogabbro
4628,432284.691263,7.396923e+06,1.0,W,5.0,0.000000,43.051680,24.634739,monzonite
4643,439758.156320,7.402305e+06,1.0,W,5.0,0.000000,46.011588,23.432545,monzodiorite monzogabbro
